In [2]:
import os
import pandas as pd

def load_predictions_data():
    predictions_path=os.path.join("..", "datasets", "predictions_prod_smallest.csv")
    return pd.read_csv(predictions_path)

In [3]:
data = load_predictions_data()

/Users/emaldonado/ml/env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [7]:
data[data['predictions_boarding_status'].notnull()]

,predictions_id,predictions_trip_id,predictions_arrival_time,predictions_boarding_status,predictions_departure_time,predictions_stop_id,predictions_stop_sequence,predictions_stops_away,predictions_vehicle_event_id,predictions_file_timestamp,predictions_route_id,predictions_vehicle_id,predictions_direction_id
26,648188038,39988632-20:30-FKenmoreStMaryC,1.553864e+09,Stopped 6 stops away,1.553864e+09,70157,100,6.0,10213965,1553862833,Green-C,G-10076,0
69,615671268,39783408,1.553249e+09,Stopped 1 stop away,1.553249e+09,70040,10,1.0,9662499,1553249378,Blue,B-545C10BB,1
83,648081068,ADDED-1553782534,1.553862e+09,Stopped 4 stops away,1.553862e+09,70202,40,4.0,10212573,1553861692,Green-E,G-10139,0
84,648167117,ADDED-1553782534,1.553863e+09,Stopped 2 stops away,1.553863e+09,70202,40,2.0,10212573,1553862592,Green-E,G-10139,0
100,648193708,39988632-20:30-FKenmoreStMaryC,1.553864e+09,Stopped 5 stops away,1.553864e+09,70159,90,5.0,10213765,1553862894,Green-C,G-10076,0
101,648193709,39988632-20:30-FKenmoreStMaryC,1.553864e+09,Stopped 6 stops away,1.553864e+09,70157,100,6.0,10213965,1553862894,Green-C,G-10076,0
139,648199348,39988632-20:30-FKenmoreStMaryC,1.553864e+09,Stopped 5 stops away,1.553864e+09,70159,90,5.0,10213765,1553862952,Green-C,G-10076,0
140,648199349,39988632-20:30-FKenmoreStMaryC,1.553864e+09,Stopped 6 stops away,1.553864e+09,70157,100,6.0,10213965,1553862952,Green-C,G-10076,0
341,644223024,40132889-L,1.553793e+09,Stopped 3 stops away,1.553793e+09,70098,40,3.0,10148828,1553792572,Red,R-545C3042,1
343,644226980,40132889-L,1.553793e+09,Stopped 3 stops away,1.553793e+09,70098,40,3.0,10148828,1553792632,Red,R-545C3042,1


In [8]:
data.head()

,predictions_id,predictions_trip_id,predictions_arrival_time,predictions_boarding_status,predictions_departure_time,predictions_stop_id,predictions_stop_sequence,predictions_stops_away,predictions_vehicle_event_id,predictions_file_timestamp,predictions_route_id,predictions_vehicle_id,predictions_direction_id
0,648187443,39783376,1.553863e+09,NaN,1.553863e+09,70040,10,1.0,10212508,1553862833,Blue,B-545C365A,1
1,648187455,39988585-20:30-FKenmoreStMaryC,1.553863e+09,NaN,1.553863e+09,70203,620,1.0,10212521,1553862833,Green-C,G-10142,1
2,648187514,40033949,1.553864e+09,NaN,1.553864e+09,70007,30,8.0,10214673,1553862833,Orange,O-545C36A0,1
3,648187604,ADDED-1553782585,1.553864e+09,NaN,1.553864e+09,70156,580,9.0,10214713,1553862833,Green-D,G-10152,1
4,648187732,40033948,1.553863e+09,NaN,1.553863e+09,70005,20,2.0,10213047,1553862833,Orange,O-545C364E,1


In [ ]:
# Keep: predictions_arrival_time, predictions_departure_time, predictions_stop_id (1-hot), predictions_direction_id
# Lose: predictions_id, predictions_trip_id, predictions_stop_sequence, predictions_vehicle_event_id, predictions_vehicle_id
# Maybe someday: predictions_stops_away, predictions_route_id (1-hot)
# Transform: is_stopped (from boarding status), predictions_file_timestamp (to get time of day / day of week)

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime
import numpy as np

class DropColumnsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, include_stops_away=False, include_route_id=False):
        self.include_stops_away = include_stops_away
        self.include_route_id = include_route_id
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.drop(["predictions_id", 
                       "predictions_trip_id", 
                       "predictions_stop_sequence",
                       "predictions_vehicle_event_id",
                       "predictions_vehicle_id",],
                      axis=1)

# Transform: is_stopped (from boarding status), predictions_file_timestamp (to get time of day / day of week)

class IsStoppedTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        result = X.copy()
        result['is_stopped'] = X['predictions_boarding_status'].notnull()
        return result.drop('predictions_boarding_status', axis=1)

class DayOfWeekTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        predictions_file_timestamp_ix = 9
        new_data = X.copy()
        predictions_day_of_week = new_data.apply(
            lambda x: datetime.fromtimestamp(x[predictions_file_timestamp_ix]).weekday(),
            axis=1
        )
        new_data.drop('predictions_file_timestamp', axis=1)
        new_data['predictions_day_of_week'] = predictions_day_of_week
        return new_data
        

In [ ]:
is_stopped_transform = IsStoppedTransformer()

is_stopped_transform.transform(new_data)

In [ ]:
new_data = data.copy()
is_stopped_transform.transform(new_data)

In [5]:
day_of_week_transformer = DayOfWeekTransformer()
new_data = data.copy()
new_data_transformed = day_of_week_transformer.transform(new_data)
new_data_transformed.head()

,predictions_id,predictions_trip_id,predictions_arrival_time,predictions_boarding_status,predictions_departure_time,predictions_stop_id,predictions_stop_sequence,predictions_stops_away,predictions_vehicle_event_id,predictions_file_timestamp,predictions_route_id,predictions_vehicle_id,predictions_direction_id,predictions_day_of_week
0,648187443,39783376,1.553863e+09,NaN,1.553863e+09,70040,10,1.0,10212508,1553862833,Blue,B-545C365A,1,4
1,648187455,39988585-20:30-FKenmoreStMaryC,1.553863e+09,NaN,1.553863e+09,70203,620,1.0,10212521,1553862833,Green-C,G-10142,1,4
2,648187514,40033949,1.553864e+09,NaN,1.553864e+09,70007,30,8.0,10214673,1553862833,Orange,O-545C36A0,1,4
3,648187604,ADDED-1553782585,1.553864e+09,NaN,1.553864e+09,70156,580,9.0,10214713,1553862833,Green-D,G-10152,1,4
4,648187732,40033948,1.553863e+09,NaN,1.553863e+09,70005,20,2.0,10213047,1553862833,Orange,O-545C364E,1,4
